# Profiling and Optimising C

In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the C-Debugging directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/C-Profiling/"))

## 1. Appending to a `List`
[test_append.c](./test_append.c) has some simple code to build a large list of square numbers.

We can compile it with optimisations turned on below:

In [ ]:
!gcc test_append.c list.c -o test_append -g -Wall -Wextra -Wpedantic -O3

The simplest profiling tool we can use is `time`. This just tells us how long the program took to run:

In [ ]:
!time ./test_append

`time` can be useful as a quick check, but timing can vary between individual runs, and a proper profiling tool will give us more useful information.

### 1.1 `gprof`
One widely available option is `gprof`. `gprof` tracks how many times each function was called, and periodically samples the program counter to estimate the time spent in each function.
To use it, we need to compile with the `-pg` flag. We also disable function reordering and inlining to make sure we can see the callgraph.

In [ ]:
!gcc test_append.c list.c -o pg_append -g -pg -Wall -Wextra -Wpedantic -O3 -fno-reorder-functions -fno-inline

This adds extra profiling code to our binary. After compiling, run the application as normal to generate a `gmon.out` file, and then view the profile information with the `gprof` command.

In [ ]:
!./pg_append && gprof ./pg_append

Since timing is based on periodic sampling it can vary significantly between runs. Accuracy can be improved by running multiple times and combining outputs with the `-s` flag.

For example, the block below runs `pg_append` ten times to create ten different profile output files. We set the `GMON_OUT_PREFIX` environment variable so that each output file has a unique name (of the format `gmon.out.<pid>` where `<pid>` is the process ID).

In [ ]:
!rm gmon.out.*; for i in {1..10}; do GMON_OUT_PREFIX="gmon.out" ./pg_append; done

Now we can combine the profiles into a single summary file (`gmon.sum`)

In [ ]:
!gprof -s ./pg_append gmon.out.*

and then analyse the result

In [ ]:
!gprof ./pg_append gmon.sum

The profile makes it clear that most of the time is spent in the `list_append` function, but it's not so clear what in particular is slow about it.

We can get extra information using the `callgrind` tool of `valgrind`.

### 1.2 `callgrind`
The `callgrind` tool will count the number of instructions executed for each line of the program, giving reproducible profiling information. This can be very useful, but will significantly increase execution time. The instruction count also does not directly correspond to execution time, since for example an I/O instruction could take significantly longer under memory pressure, so `callgrind` is best used in combination with timing tools (like `gprof`).

In this case, we don't want the `-pg` compiler flag, since that will add overhead that we don't need, but we do still need the `-g` flag to include debug symbols (needed to link instructions to lines in the source code). Note that with optimisations enabled, some lines might not have any associated instructions.

In [ ]:
!gcc test_append.c list.c -o test_append -g -Wall -Wextra -Wpedantic -O3 -fno-reorder-functions -fno-inline

We can run `callgrind` with the block below (this will take a while):

In [ ]:
!rm callgrind.out.*; valgrind --tool=callgrind ./test_append

This creates a `callgrind.out.<pid>` file which we can analyse with `callgrind_annotate`:

In [ ]:
!callgrind_annotate callgrind.out.*

This tells us that the vast majority of instructions are happening inside the call to `realloc`. We already know that `list_append` is the bottleneck, and now we know that most of the work in that function is done in `realloc`, so we can use this information to decide on an optimisation strategy.

Try improving the code so that building the list of squared numbers is faster.


#### HINTS

* Allocating memory is generally slow. `realloc` has some optimisations that will sometimes make it faster, but it's not guaranteed.
* Our program knows ahead of time that there will be exactly `N` items in the list, but it currently calls `realloc` every time a new item is appended.
* By changing how `List` manages its memory, we can reduce the number of calls to `realloc` to avoid unnecessary work.

If you just want to see the solution, you can look at [test_append_optimised.c](./test_append_optimised.c), [list_optimised.c](./list_optimised.c) and [list_optimised.h](./list_optimised.h).

## 2. Matrix Multiplication

Matrix-matrix multiplication gives us a chance to examine some other optimisation techniques.
Take a look at [matmul.c](./matmul.c). It repeats a 500 x 500 matrix multiplication 100 times to test the `matmul` function.

It can be compiled with the command below. From a VDI session, try profiling it with Arm MAP and using the profile information to speed it up. For larger applications, you may need some extra compile flags to get the most information. See [the documentation](https://developer.arm.com/documentation/101136/2100/MAP/Get-started-with-MAP/Prepare-a-program-for-profiling?lang=en) for details.

You may also find [c.godbolt.org](https://c.godbolt.org) handy for fine-grained analysis of the instructions output by the compiler. With some practice, this can be useful for checking things like auto-vectorisation. Don't forget to set the `-O3` flag, as well as `-mavx2` to tell the compiler that AVX2 vector instructions are available (or `-mavx512` for 512 bit vector instructions, or specify the exact CPU architecture like `-march=sapphirerapids`).

In [ ]:
!gcc matmul.c -o matmul -g -Wall -Wextra -Wpedantic -lm -O3

#### HINTS

There are a number of inefficiencies in the code related to:
* Memory layout
* Cache friendliness
* Compiler friendliness (mainly for auto-vectorisation)

Consider optimisations such as:
* Avoiding pointer chasing (it's faster to just follow one pointer than to follow one and then annother)
* Using contiguous memory (it's generally faster to work with a single large block of memory instead of many smaller blocks)
* Tweaking loop ordering (accessing elements in the same order they're stored in memory can give better cache performance and help the compiler to auto-vectorise)
* Using the `restrict` keyword to promise to the compiler that the `A`, `B` and `C` matrices don't overlap in memory (the compiler won't assume this by default, which can hinder some optimisations like auto-vectorisation).
  - `restrict` can be used with standard pointers as `double *restrict my_ptr;` (it applies to the left like `const`) and with arrays as `double array[restrict N]`
 
See [matmul_optimised.c](./matmul_optimised.c) for an example solution. Note that further optimisation is possible, and libraries such as BLAS and BLIS are significantly faster.